In [1]:
import os
import pandas as pd
import torch

import importlib
import pickle
import re

base_path = os.getcwd()
data_path = os.path.join(base_path, "data")
question_path = os.path.join(data_path, "questions")
python_path = os.path.join(data_path, "python")

In [2]:
import ProblemSetReader
importlib.reload(ProblemSetReader)
problem_set_reader = ProblemSetReader.ProblemSetReader()
df = pd.read_pickle("./data/euler_df.pkl")

In [3]:
df

,idx,question,answer,python_code
0,1,If we list all the natural numbers below $10$ ...,2.331680e+05,\n\ndef compute():\n\tans = sum(x for x in ran...
1,2,Each new term in the Fibonacci sequence is gen...,4.613732e+06,\n\ndef compute():\n\tans = 0\n\tx = 1 # Repr...
2,3,"The prime factors of $13195$ are $5, 7, 13$ an...",6.857000e+03,\nimport math\n\n\ndef compute():\n\tn = 60085...
3,4,A palindromic number reads the same both ways....,9.066090e+05,\n\ndef compute():\n\tans = max(i * j\n\t\tfor...
4,5,$2520$ is the smallest number that can be divi...,2.327926e+08,\nimport math\n\n\ndef compute():\n\tans = mat...
...,...,...,...,...
186,451,\nConsider the number $15$.\nThere are eight p...,1.536511e+14,"\nimport array, itertools, math\n\n\ndef compu..."
187,500,The number of divisors of $120$ is $16$.\nIn f...,3.540728e+07,"\nimport eulerlib, heapq\n\n\ndef compute():\n..."
188,518,Let $S(n) = \sum a + b + c$ over all triples $...,1.003157e+14,"\nimport eulerlib, math, itertools\n\n\ndef co..."
189,549,\nThe smallest number $m$ such that $10$ divid...,4.760015e+14,\nimport itertools\n\n\ndef compute():\n\tLIMI...


# Test GPT

In [125]:
import GPTAgent
importlib.reload(GPTAgent)
gpt_agent = GPTAgent.GPTAgent()

In [126]:
response_string = gpt_agent.generate_chain_of_thoughts(problem_set_reader.get_question(125))
procedures = response_string.split("\n")
procedures

['1. Define the problem statement and understand the required input/output.',
 '2. Generate palindromic numbers from 1 to $10^8$.',
 '3. For each palindromic number, find all possible combinations of consecutive squares that sum up to the palindromic number.',
 '4. Verify if the sum of consecutive squares is valid for each palindromic number, excluding the case of $1 = 0^2 + 1^2$.',
 '5. Print the sum of all numbers less than $10^8$ that are both palindromic and can be written as the sum of consecutive squares.']

In [130]:
from g4f.client import Client

client = Client()
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "system", "content": "Only respond in English."},
              {"role": "user", "content": "How to make a cup of tea?"}, 
              {"role": "assistant", "content": "Simply boil water and add tea."}, 
              {"role": "user", "content": "What is the last question I asked you?"}],
)
response.choices[0].message.content

"I'm sorry, but I don't have the ability to recall past interactions or questions from previous conversations. Each session with me is independent, and I treat every interaction as a new encounter. How may I assist you today?\n"

In [127]:
messages = []

for step_description in procedures:
    prompt = f"Generate at most two line of python code (packed by ```python {{python_code}}```) for this step (and only this step): {step_description}. \
        Directly output the code without explaination, output space to indent if necessary"

    messages.append({"role": "user", "content": prompt})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    ret_string = response.choices[0].message.content
    print(ret_string)
    messages.append({"role": "assistant", "content": ret_string})

```
python
problem_statement = input("请输入问题描述：")
required_input_output = input("请输入所需输入输出：")
```
```python
for i in range(1, 10**8 + 1):
    if str(i) == str(i)[::-1]:
        print(i)
```
```python
import itertools
for i in range(1, 10**8 + 1):
    if str(i) == str(i)[::-1]:
        for r in range(1, int(i**0.5) + 1):
            for combo in itertools.combinations(range(1, r + 1), r):
                if sum(j**2 for j in combo) == i:
                    print(f"{i} = {'+'.join(str(j**2) for j in combo)}")
```
```python
for i in range(1, 10**8 + 1):
    if str(i) == str(i)[::-1] and i!= 2 and any(sum(j**2 for j in range(a, b)) == i for a in range(1, int(i**0.5) + 1) for b in range(a, int(i**0.5) + 2)):
        print(i)
```
```python
sum_of_palindromes = sum(i for i in range(1, 10**8 + 1) if str(i) == str(i)[::-1] and any(sum(j**2 for j in range(a, b)) == i for a in range(1, int(i**0.5) + 1) for b in range(a, int(i**0.5) + 2)) and i != 1)
print(sum_of_palindromes)
```


In [128]:
raw_code = [messages['content'] for messages in messages if messages['role']=='assistant']
raw_code

['```\npython\nproblem_statement = input("请输入问题描述：")\nrequired_input_output = input("请输入所需输入输出：")\n```',
 '```python\nfor i in range(1, 10**8 + 1):\n    if str(i) == str(i)[::-1]:\n        print(i)\n```',
 '```python\nimport itertools\nfor i in range(1, 10**8 + 1):\n    if str(i) == str(i)[::-1]:\n        for r in range(1, int(i**0.5) + 1):\n            for combo in itertools.combinations(range(1, r + 1), r):\n                if sum(j**2 for j in combo) == i:\n                    print(f"{i} = {\'+\'.join(str(j**2) for j in combo)}")\n```',
 '```python\nfor i in range(1, 10**8 + 1):\n    if str(i) == str(i)[::-1] and i!= 2 and any(sum(j**2 for j in range(a, b)) == i for a in range(1, int(i**0.5) + 1) for b in range(a, int(i**0.5) + 2)):\n        print(i)\n```',
 '```python\nsum_of_palindromes = sum(i for i in range(1, 10**8 + 1) if str(i) == str(i)[::-1] and any(sum(j**2 for j in range(a, b)) == i for a in range(1, int(i**0.5) + 1) for b in range(a, int(i**0.5) + 2)) and i != 1)\nprint

In [129]:
pattern = r"```python(.*?)```"
code_cleansed = [match.group(1) for match in re.finditer(pattern, '\n'.join(raw_code), re.DOTALL)]
code_concat = "".join(code_cleansed)
print(code_concat)


for i in range(1, 10**8 + 1):
    if str(i) == str(i)[::-1]:
        print(i)

import itertools
for i in range(1, 10**8 + 1):
    if str(i) == str(i)[::-1]:
        for r in range(1, int(i**0.5) + 1):
            for combo in itertools.combinations(range(1, r + 1), r):
                if sum(j**2 for j in combo) == i:
                    print(f"{i} = {'+'.join(str(j**2) for j in combo)}")

for i in range(1, 10**8 + 1):
    if str(i) == str(i)[::-1] and i!= 2 and any(sum(j**2 for j in range(a, b)) == i for a in range(1, int(i**0.5) + 1) for b in range(a, int(i**0.5) + 2)):
        print(i)

sum_of_palindromes = sum(i for i in range(1, 10**8 + 1) if str(i) == str(i)[::-1] and any(sum(j**2 for j in range(a, b)) == i for a in range(1, int(i**0.5) + 1) for b in range(a, int(i**0.5) + 2)) and i != 1)
print(sum_of_palindromes)

